In [11]:

import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
from scipy.interpolate import splev, splprep
from torch.utils.data import ConcatDataset, DataLoader,Dataset
import random
import os
from PIL import Image,ImageDraw
import sys
sys.path.append('../')
import config


def get_spline_pts(leaf_pts, num_pts=10):
    """
    Generate spline points based on the given leaf points.

    Parameters:
        leaf_pts (array_like): The input leaf points.
        num_pts (int, optional): The number of points to generate on the spline. Default is 10.

    Returns:
        array_like: The spline points.

    Notes:
        - The function uses `splprep` to obtain a B-spline representation of the curve that passes through the given points.
        - The B-spline representation consists of three elements: knots, coefficients, and the degree of the spline.
        - The knots define the curve, the coefficients are the weights of the curve, and the degree is the degree of the polynomial that defines the curve.
        - The function uses `splev` to evaluate the B-spline representation and obtain the points on the curve.

    """
    tck, u = splprep(leaf_pts.T, s=0, k=min(3, len(leaf_pts) - 1))
    x = np.linspace(u.min(), u.max(), num_pts)
    y = np.array(splev(x, tck, der=0))
    return y.T





def plot_images_two_dataloaders(loaders, num_images=2):
    plt.rcParams["figure.figsize"] = [10, 5]  # Set the figure size

    for i, (loader1_item, loader2_item) in enumerate(zip(loaders[0], loaders[1])):
        images1 = loader1_item["image"]
        landmarks1_lateral = loader1_item["landmarks"]["leaflet_lateral"]
        landmarks1_septal = loader1_item["landmarks"]["leaflet_septal"]

        images2 = loader2_item["image"]
        landmarks2_lateral = loader2_item["landmarks"]["leaflet_lateral"]
        landmarks2_septal = loader2_item["landmarks"]["leaflet_septal"]

        # Plot images and landmarks from both loaders
        for (
            image1,
            landmark1_lateral,
            landmark1_septal,
            image2,
            landmark2_lateral,
            landmark2_septal,
        ) in zip(
            images1,
            landmarks1_lateral,
            landmarks1_septal,
            images2,
            landmarks2_lateral,
            landmarks2_septal,
        ):
            # Create a subplot for image and landmarks
            fig, (ax1, ax2) = plt.subplots(1, 2)

            # Set the figure size for each subplot
            ax1.figure.set_size_inches(7, 5)
            ax2.figure.set_size_inches(7, 5)

            # Plot image and landmarks from the first loader
            ax1.imshow(image1, cmap="gray")
            ax1.axis("off")
            ax1.scatter(
                landmark1_lateral[:, 0], landmark1_lateral[:, 1], c="red", s=1
            )  # Set smaller point size
            ax1.scatter(
                landmark1_septal[:, 0], landmark1_septal[:, 1], c="blue", s=1
            )  # Set smaller point size

            # Plot image and landmarks from the second loader
            ax2.imshow(image2, cmap="gray")
            ax2.axis("off")
            ax2.scatter(
                landmark2_lateral[:, 0], landmark2_lateral[:, 1], c="red", s=1
            )  # Set smaller point size
            ax2.scatter(
                landmark2_septal[:, 0], landmark2_septal[:, 1], c="blue", s=1
            )  # Set smaller point size

            plt.show()

            # Check if the desired number of images have been plotted
            num_images -= 1
            if num_images == 0:
                return




# Define a function to create a mask from an image and a dictionary
def create_mask_polygon(image, dictionary,no_of_points=40):
  # Extract the 'leaflet_septal' key values and convert them to a list of tuples
  coordinates = dictionary['leaflet_septal']
  coordinates=get_spline_pts(coordinates,no_of_points)
  coordinates = [tuple(p) for p in coordinates]
  

  # Create a blank image with the same shape as the input image
  mask = Image.new('L', (image.shape[1], image.shape[0]), 0)

  # Draw a polygon on the blank image using the coordinates
  ImageDraw.Draw(mask).polygon(coordinates, outline=1, fill=1)

  # Convert the blank image to a numpy array
  mask = np.array(mask)

    # # Optionally, display the images and the masks
    # for i in range(len(x)):
    # plt.figure()
    # plt.subplot(1,2,1)
    # plt.imshow(x[i], cmap='gray')
    # plt.title('Image')
    # plt.subplot(1,2,2)
    # plt.imshow(masks[i], cmap='gray')
    # plt.title('Mask')
    # plt.show()


  # Return the mask as a binary array
  return mask.astype(bool)



# def create_mask_line(image, annotation, no_of_points=40,thickness=2):
#     """
#     Plot points on an image using a binary mask with added thickness.

#     Parameters:
#         image (ndarray): NumPy array containing the image.
#         coordinates_array (ndarray): NumPy array containing the coordinates.
#         thickness (int): Thickness of the plotted points (default is 3).

#     Returns:
#         None (displays the image and mask).
#     """
#     coordinates = annotation.get('leaflet_septal')
#     coordinates=get_spline_pts(coordinates,no_of_points)

#     # Convert floating-point coordinates to integers
#     coordinates_array = coordinates.astype(int)

#     # Create a blank mask with the same size as the image
#     mask = np.zeros_like(image)

#     # Plot the points on the mask with added thickness
#     for x, y in coordinates_array:
#         mask[max(0, y - thickness):min(mask.shape[0], y + thickness +1),
#              max(0, x - thickness):min(mask.shape[1], x + thickness +1)] = 1
#     return mask.astype(bool)

def create_mask_line(image, annotation, no_of_points=40, thickness=2):
    """
    Plot points on an image using a binary mask with added thickness.

    Parameters:
        image (ndarray): NumPy array containing the image.
        leaflet_septal (array_like): NumPy array containing the septal leaflet coordinates.
        leaflet_lateral (array_like): NumPy array containing the lateral leaflet coordinates.
        no_of_points (int): Number of points to generate on the B-spline (default is 40).
        thickness (int): Thickness of the plotted points (default is 2).

    Returns:
        ndarray: The final mask containing the combined points from both leaflet sets.
    """



    coordinates_septal = get_spline_pts(annotation.get('leaflet_septal'), no_of_points)
    coordinates_lateral = get_spline_pts(annotation.get('leaflet_septal'), no_of_points)

    # Convert floating-point coordinates to integers
    coordinates_array_septal = coordinates_septal.astype(int)
    coordinates_array_lateral = coordinates_lateral.astype(int)

    # Create blank masks with the same size as the image
    mask_septal = np.zeros_like(image)
    mask_lateral = np.zeros_like(image)

    # Plot the points on the masks with added thickness
    for x, y in coordinates_array_septal:
        mask_septal[max(0, y - thickness):min(mask_septal.shape[0], y + thickness + 1),
                    max(0, x - thickness):min(mask_septal.shape[1], x + thickness + 1)] = 1

    for x, y in coordinates_array_lateral:
        mask_lateral[max(0, y - thickness):min(mask_lateral.shape[0], y + thickness + 1),
                     max(0, x - thickness):min(mask_lateral.shape[1], x + thickness + 1)] = 1

    # Combine the masks (OR operation) to get the final mask
    final_mask = np.logical_or(mask_septal, mask_lateral)

    return final_mask.astype(bool)


# Define a custom dataset class
class ImageMaskDataset(Dataset):
  def __init__(self, images, masks, transform=None):
    # Store the images and masks as attributes
    self.images = images
    self.masks = masks
    # Store the transform as an attribute
    self.transform = transform

  def __len__(self):
    # Return the length of the list of images or masks
    return len(self.images)

  def __getitem__(self, idx):
    # Get an image and a mask at a given index
    image = self.images[idx]
    mask = self.masks[idx]
    # Convert them to PIL Image objects
    image = Image.fromarray(image)
    mask = Image.fromarray(mask)
    # Apply the transform if any
    if self.transform:
      image = self.transform(image)
      mask = self.transform(mask)
    # Return a tuple of image and mask
    return image, mask


def create_dataloaders_with_masks_polygons(training_loader, val_loader, testing_loader,no_of_points):
  """Creates dataloaders with masks for training, validation, and testing.

  Args:
    training_loader: The training dataloader.
    val_loader: The validation dataloader.
    testing_loader: The testing dataloader.

  Returns:
    A tuple of dataloaders with masks.
  """
  masks_training = []
  masks_val = []
  masks_testing = []

  for image, dictionary in zip(training_loader.dataset.images, training_loader.dataset.landmarks):
    mask = create_mask_polygon(image, dictionary,no_of_points)
    masks_training.append(mask)


  for image, dictionary in zip(val_loader.dataset.images, val_loader.dataset.landmarks):
    mask = create_mask_polygon(image, dictionary,no_of_points)
    masks_val.append(mask)

  for image, dictionary in zip(testing_loader.dataset.images, testing_loader.dataset.landmarks):
    mask = create_mask_polygon(image, dictionary,no_of_points)
    masks_testing.append(mask)

  train=ImageMaskDataset(training_loader.dataset.images,masks_training)
  val=ImageMaskDataset(val_loader.dataset.images,masks_val)
  testing=ImageMaskDataset(testing_loader.dataset.images,masks_testing)

  return (train, val, testing)

def create_dataloaders_with_masks_line(training_loader, val_loader, testing_loader,no_of_points=10,mask_thickness=2):
  """Creates dataloaders with masks for training, validation, and testing.

  Args:
    training_loader: The training dataloader.
    val_loader: The validation dataloader.
    testing_loader: The testing dataloader.

  Returns:
    A tuple of dataloaders with masks.
  """
  masks_training = []
  masks_val = []
  masks_testing = []

  for image, dictionary in zip(training_loader.dataset.images, training_loader.dataset.landmarks):
    mask = create_mask_line(image, dictionary,no_of_points,mask_thickness)
    masks_training.append(mask)


  for image, dictionary in zip(val_loader.dataset.images, val_loader.dataset.landmarks):
    mask = create_mask_line(image, dictionary,no_of_points,mask_thickness)
    masks_val.append(mask)

  for image, dictionary in zip(testing_loader.dataset.images, testing_loader.dataset.landmarks):
    mask = create_mask_line(image, dictionary,no_of_points,mask_thickness)
    masks_testing.append(mask)

  train=ImageMaskDataset(training_loader.dataset.images,masks_training)
  val=ImageMaskDataset(val_loader.dataset.images,masks_val)
  testing=ImageMaskDataset(testing_loader.dataset.images,masks_testing)

  return (train, val, testing)


def save_images_masks(dataset, data_type, mask_type, folder_path):
    """
    dataset: an instance of the ImageMaskDataset class
    data_type: a string representing the type of data, e.g. 'train', 'test', or 'val'
    mask_type: line or polygon
    folder_path: location to save the segmentation files
    """
    mask_subfolder = 'segmentation_lines' if mask_type == 'line' else 'segmentation_polygons'
    no_of_points = config.no_of_points

    if mask_type == 'line':
        mask_thickness = config.mask_thickness
        folder_path = f"{folder_path}/{mask_subfolder}_{no_of_points}_{mask_thickness}/"
    else:
        folder_path = f"{folder_path}/{mask_subfolder}_{no_of_points}/"

    # Create the directories
    os.makedirs(f'{folder_path}{data_type}/images', exist_ok=True)
    os.makedirs(f'{folder_path}{data_type}/masks', exist_ok=True)

    # Save the images and masks
    for i in range(len(dataset)):
        image, mask = dataset.images[i], dataset.masks[i]
        image = Image.fromarray(image)
        mask = Image.fromarray(mask)
        image.save(f'{folder_path}{data_type}/images/image_{i}.png')
        mask.save(f'{folder_path}{data_type}/masks/image_{i}.png')


def save_images_and_masks_in_folders(train_loader, val_loader, test_loader, mask_type, base_folder_path="../"):
    # Save images and masks for train
    save_images_masks(train_loader, 'train', mask_type, base_folder_path)

    # Save images and masks for val
    save_images_masks(val_loader, 'val', mask_type, base_folder_path)

    # Save images and masks for test
    save_images_masks(test_loader, 'test', mask_type, base_folder_path)







ModuleNotFoundError: No module named 'src'

In [9]:
import matplotlib.pyplot as plt
import numpy as np

import sys
sys.path.append('../')
import config as cfg
import data_loader as dl
import utils
# import wandb


import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np


ModuleNotFoundError: No module named 'config'

In [ ]:
training_loader = dl.get_data_loader(
    mode="train", batch_size=8, num_workers=0
    )
val_loader = dl.get_data_loader(
        mode="val", batch_size=8, num_workers=0)
testing_loader = dl.get_data_loader(
        mode="test", batch_size=8, num_workers=0
    )


In [ ]:
len(training_loader.dataset)

In [ ]:
image1=training_loader.dataset.images[0]
mask1=training_loader.dataset.landmarks[0].get("leaflet_septal")

In [ ]:
coordinates=utils.get_spline_pts(mask1,30)

coordinates

## get final dataloaders with image and masks

In [ ]:
train,val,test=utils.create_dataloaders_with_masks_line(training_loader, val_loader, testing_loader, no_of_points=30, mask_thickness=1)

In [ ]:
print(f" size of dataoader : -> \n Train {len(train)} \n Val {len(val)} \n Test {len(test)}")

In [ ]:
utils.save_images_and_masks_in_folders(train,val,test,'masks',base_folder_path="../..")

In [ ]:
def plot_image_mask(image, mask):
  # Create a figure with two subplots
  fig, axs = plt.subplots(1, 2, figsize=(10, 5))

  # Plot the image in the first subplot
  axs[0].imshow(image, cmap='gray')
  axs[0].set_title('Image')

  # Plot the mask in the second subplot
  axs[1].imshow(mask, cmap='gray')
  axs[1].set_title('Mask')

  # Show the plot
  plt.show()


In [ ]:
train,val,test=utils.create_dataloaders_with_masks_line(training_loader, val_loader, testing_loader,no_of_points=30)
image1=train.images[0]
mask1=train.masks[0]
plot_image_mask(image1,mask1)

In [ ]:
train,val,test=utils.create_dataloaders_with_masks_line(training_loader, val_loader, testing_loader,no_of_points=50,thickness=1)
image1=train.images[0]
mask1=train.masks[0]
plot_image_mask(image1,mask1)

In [ ]:
train,val,test=utils.create_dataloaders_with_masks_line(training_loader, val_loader, testing_loader,no_of_points=50,thickness=2)
image1=train.images[0]
mask1=train.masks[0]
plot_image_mask(image1,mask1)

In [ ]:
train,val,test=utils.create_dataloaders_with_masks_line(training_loader, val_loader, testing_loader,no_of_points=10,thickness=2)
image1=train.images[0]
mask1=train.masks[0]
plot_image_mask(image1,mask1)

In [ ]:
train,val,test=utils.create_dataloaders_with_masks_line(training_loader, val_loader, testing_loader,no_of_points=10,thickness=1)
image1=train.images[0]
mask1=train.masks[0]
plot_image_mask(image1,mask1)

### Trying all combinations


In [ ]:
import os

def plot_image_mask_combinations(no_of_points_list, thickness_list,save=True):
    """
    Plot images and masks for different combinations of the number of points and thickness.

    Parameters:
        no_of_points_list (list): List of integers representing the number of points to use.
        thickness_list (list): List of float values representing the thickness of points.
        save (bool): Whether to save the plots or not.

    Returns:
        None (displays the images and masks).
    """


    for no_of_points in no_of_points_list:
        for thickness in thickness_list:
            train, _, _ = utils.create_dataloaders_with_masks_line(training_loader, val_loader, testing_loader, no_of_points, thickness)

            image1 = train.images[0]
            mask1 = train.masks[0]

            # Create the plot
            fig, ax = plt.subplots(1, 2, figsize=(10, 5))

            # Plot the image
            ax[0].imshow(image1, cmap='gray')
            ax[0].set_title(f'Image (Points: {no_of_points}, Thickness: {thickness})')

            # Plot the mask
            ax[1].imshow(mask1, cmap='gray')
            ax[1].set_title('Mask')

            # Add point and thickness information as annotations
            ax[0].text(0.5, -0.1, f'Points: {no_of_points}', fontsize=10, ha='center', transform=ax[0].transAxes)
            ax[0].text(0.5, -0.2, f'Thickness: {thickness}', fontsize=10, ha='center', transform=ax[0].transAxes)

            # Show the plot
            if save==True:
                os.makedirs("plots/",exist_ok=True)
                filename = f"plots/plot_points_{no_of_points}_thickness_{thickness}.png"
                plt.savefig(filename)
                plt.show()
            else:
                plt.show()    

            

# Example usage:
# Replace the lists with your desired combinations of no_of_points and thickness
no_of_points_list = [10,20,30,40,50,]
thickness_list = [0, 1,2, 3]
plot_image_mask_combinations(no_of_points_list, thickness_list)




## saves files for polygons

In [ ]:
train.images[0].shape

In [ ]:
train.masks[0].shape

In [ ]:
train.masks[6]

In [ ]:
## find pixel values where train.masks[6] is true 
## and print those pixel values
np.where(train.masks[6])


In [ ]:
plot_image_mask(train.images[5],train.masks[6])